# Preparación de los datos para tablero Encuesta Prácticas Deportivas y Calidad de Vida 2024

Secretaría de Cultura, Recreación y Deporte | Dirección Observatorio y Gestión del Conocimiento Cultural | Sistemas de Información y Narrativas

30 de abril de 2025

- Este notebook muestra el proceso de lectura, procesamiento y preparación de datos de las secciones, preguntas, variables (subpreguntas), opciones de respuesta, respuestas detalladas y extendidas de la encuesta de práctivas deportivas y calidad de vida 2024, como módulo de la Encuesta Bienal de Culturas.
- Con estos datos se construye el tablero de visualización de datos en Power Bi, disponible en el siguiente enlace [Tablero EBC Prácticas Deportivas y Calidad de Vida 2024](https://app.powerbi.com/view?r=eyJrIjoiNjBkMjFjMjktZWIwZi00ZjA5LWEwNGUtODVmYzUyODFlNDYyIiwidCI6IjRmNzkzOWM3LWFhNjAtNDliZC05YjdiLTZmODFjMzdkMWIzNyJ9&pageName=69b48348ad3ee2b5bec5)
- Los datos completos y archivos de construcción están disponibles en la siguiente carpeta de Google Drive [m182 EBC Prácticas deportivas 2024](https://drive.google.com/drive/folders/13Ut2WIxtUHiPrdhqGPUhFwG2EFzScwtm).

In [1]:
import pandas as pd

In [2]:
codigo_medicion = 'm182'

In [3]:
df_respuestas = pd.read_excel(f'{codigo_medicion}_datos.xlsx',sheet_name='datos')

In [4]:
# Obtener las columnas de tipo float
columnas_float = df_respuestas.select_dtypes(include=['float']).columns.tolist()

# Iterar sobre las columnas de tipo float y mostrarlas
""" for columna in columnas_float:
    print(columna) """

' for columna in columnas_float:\n    print(columna) '

In [5]:
# Diccionario de datos, detalle de columnas
df_variables = pd.read_excel(f'{codigo_medicion}_datos.xlsx',sheet_name='variables')

In [6]:
# Variables para derretir en detalle
#df_variables_melt = df_variables[(df_variables['derretir'].notnull() & (df_variables['derretir'] != "") & (df_variables['tipo'] == 50))]
df_variables_melt = df_variables[(df_variables['derretir'].notnull() & (df_variables['derretir'] != ""))]

In [7]:
# Filtrar las filas donde la columna 'grupo' contiene la cadena "SiNo,"
to_boolean_variables = df_variables[df_variables['grupo'].str.contains("SiNo,", na=False)]['codigo_variable'].tolist()
# Recorrer la lista de variables booleanas, para reemplazar los valores
for variable in to_boolean_variables:
    # Reemplazar valores en la columna correspondiente
    df_respuestas[variable] = df_respuestas[variable].apply(lambda x: "Sí" if pd.notnull(x) and str(x).strip() != "" else "No")

## Generar detalle de respuestas
Generar datos para la tabla *respuestas*

In [8]:
# Aplica la función melt en df_respuestas para "derretir" las columnas especificadas en df_variables_melt
df_respuestas_melt = pd.melt(
    df_respuestas, 
    id_vars=['encuestado_id', 'FACTOR', 'EDAD'],  # Estas son las columnas que queremos conservar
    value_vars=df_variables_melt['codigo_variable'],  # Estas son las columnas a derretir
    var_name='codigo_variable', 
    value_name='respuesta'
)

# Ahora hacemos el merge entre df_respuestas_melt y df_variables_melt para agregar las columnas 'indice_pregunta' e 'indice_variable'
df_respuestas_melt = df_respuestas_melt.merge(
    df_variables[['codigo_variable', 'indice_pregunta', 'indice_variable', 'pregunta']], 
    on='codigo_variable', 
    how='left'
)


# Se reemplaza el los valores nan de la columna 'respuesta' por "NS/NR"
df_respuestas_melt['respuesta'] = df_respuestas_melt['respuesta'].fillna("NS/NR")
df_respuestas_melt['respuesta'] = df_respuestas_melt['respuesta'].str.strip()

# Renombramos la columna FACTOR a factor_expansion
df_respuestas_melt.rename(columns={'FACTOR': 'factor_expansion'}, inplace=True)
df_respuestas_melt.head()

,encuestado_id,factor_expansion,EDAD,codigo_variable,respuesta,indice_pregunta,indice_variable,pregunta
0,1,1257.35490,78,Id,NaN,0,1,ID
1,2,209.11119,19,Id,NaN,0,1,ID
2,3,501.42880,23,Id,NaN,0,1,ID
3,4,396.20236,32,Id,NaN,0,1,ID
4,5,1951.11670,55,Id,NaN,0,1,ID


In [9]:
# Filtrar los registros donde indice_pregunta es igual a 92
registros_filtrados = df_respuestas_melt[df_respuestas_melt['indice_variable'] == 92]

# Mostrar la cantidad de registros
registros_filtrados_count = registros_filtrados.shape[0]
print(f"Cantidad de registros donde indice_pregunta es igual a 92: {registros_filtrados_count}")

registros_filtrados.to_excel('testing.xlsx', index=False)

Cantidad de registros donde indice_pregunta es igual a 92: 4549


In [10]:
# Extraer las columnas "pregunta" y "respuesta" del DataFrame original
df_opciones = df_respuestas_melt[['pregunta', 'respuesta']].copy()

# Eliminar espacios a la derecha e izquierda del texto en la columna 'respuesta'
df_opciones['respuesta'] = df_opciones['respuesta'].str.strip()

# Eliminar filas duplicadas
df_opciones = df_opciones.drop_duplicates()

# Opcional: Reiniciar el índice del DataFrame
df_opciones = df_opciones.reset_index(drop=True)

#df_opciones.dropna(subset=['respuesta'], inplace=True)

# Mostrar el DataFrame resultante
df_opciones.head(5)

,pregunta,respuesta
0,ID,NaN
1,H1,NaN
2,H2,NaN
3,H51,NaN
4,V1,Usme


In [11]:
# Sobre escribir archivo
from openpyxl import load_workbook

# Ruta del archivo Excel
file_path = f'{codigo_medicion}_opciones.xlsx'

# Cargar el archivo Excel existente
book = load_workbook(file_path)

# Utilizamos el engine openpyxl para abrir el archivo sin sobrescribir las otras hojas
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    # Escribimos o actualizamos la hoja de cálculo 'respuestas_detalle'
    df_opciones.to_excel(writer, index=False, sheet_name='opciones_pre')

In [12]:
df_respuestas_melt['respuesta'] = df_respuestas_melt['respuesta'].astype(str)
df_respuestas_melt.to_parquet(f'{codigo_medicion}_respuestas_detalle.parquet', index=False)